### Tree Methods with PySpark

1. Single Decision Tree
2. Random Forest
3. Gradient Boosted Tree Classifier 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trees").getOrCreate()

24/05/14 14:27:54 WARN Utils: Your hostname, aditya resolves to a loopback address: 127.0.1.1; using 192.168.29.243 instead (on interface wlo1)
24/05/14 14:27:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/14 14:27:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/14 14:27:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/14 14:27:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/14 14:27:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
df = spark.read.csv("college.csv", inferSchema=True, header=True)

In [3]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [6]:
df.show(2)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--

#### Formatting for Spark

In [7]:
# "label", "features"

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [11]:
assembler = VectorAssembler(
    inputCols = [
     'Apps',
     'Accept',
     'Enroll',
     'Top10perc',
     'Top25perc',
     'F_Undergrad',
     'P_Undergrad',
     'Outstate',
     'Room_Board',
     'Books',
     'Personal',
     'PhD',
     'Terminal',
     'S_F_Ratio',
     'perc_alumni',
     'Expend',
     'Grad_Rate'
    ],
    outputCol="features"
)

In [12]:
output = assembler.transform(df)

In [15]:
# String variables (private)
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndexer")

In [16]:
output_fixed = indexer.fit(output).transform(output)

In [17]:
df_final = output_fixed.select("features", "PrivateIndexer")

In [18]:
train, test = df_final.randomSplit([0.7, 0.3], seed=42)

#### Tree Classifiers

In [20]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline

#### Create Models

In [21]:
dtc = DecisionTreeClassifier(labelCol="PrivateIndexer", featuresCol="features")
rfc = RandomForestClassifier(labelCol="PrivateIndexer", featuresCol="features")
gbt = GBTClassifier(labelCol="PrivateIndexer", featuresCol="features")

In [22]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

#### Prediction

In [23]:
dtc_pred = dtc_model.transform(test)
rfc_pred = rfc_model.transform(test)
gbt_pred = gbt_model.transform(test)

#### Evaluation

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndexer", predictionCol="prediction", metricName="accuracy")

In [27]:
dtc_acc = evaluator.evaluate(dtc_pred)
rfc_acc = evaluator.evaluate(rfc_pred)
gbt_acc = evaluator.evaluate(gbt_pred)

In [28]:
print("-"*90)
print(f"DT Acc: {dtc_acc}")
print("-"*90)
print(f"RFC Acc: {rfc_acc}")
print("-"*90)
print(f"GBT Acc: {gbt_acc}")

------------------------------------------------------------------------------------------
DT Acc: 0.925
------------------------------------------------------------------------------------------
RFC Acc: 0.93
------------------------------------------------------------------------------------------
GBT Acc: 0.94
